In [ ]:
from copy import deepcopy
from scipy.stats import chi2
from math import sqrt, exp, log

def summary(cube):
    res = []
    tables = len(cube)
    rows = len(cube[0])
    columns = len(cube[0][0])
    for i in range(rows):
        row = []
        for j in range(columns):
            row.append(sum(cube[table][i][j] for table in range(tables)))
        res.append(row)
    return res

def marginals(table):
    res = deepcopy(table)
    columns = len(table[0])

    print(table)
    
    column_totals = [sum(row[column] for row in table) for column in range(columns)]
    row_totals = [sum(elem for elem in row) for row in table] + [sum(column_totals)]

    res.append(column_totals)
    for i, total in enumerate(row_totals):
        res[i].append(total)
    return res

def cochran_mantel_haenzsel(cube):
    # https://en.wikipedia.org/wiki/Cochran%E2%80%93Mantel%E2%80%93Haenszel_statistics
    sum_1 = 0
    sum_2 = 0
    for table in cube:
        a = table[0][0]
        b = table[0][1]
        c = table[1][0]
        d = table[1][1]

        n = a + b + c + d
        sum_1 += a - (a+b)*(a+c)/n
        sum_2 += (a+b)*(a+c)*(b+d)*(c+d)/(pow(n,3)-pow(n,2))

    obs = pow(sum_1, 2)/sum_2
    
    return obs, chi2.sf(obs, 1)

def ods_ratio(cube):
    sum_1 = 0
    sum_2 = 0
    Q = 0
    R = 0
    V = 0
    for table in cube:
        a = table[0][0]
        b = table[0][1]
        c = table[1][0]
        d = table[1][1]

        n = a + b + c + d

        sum_1 += a*d/n
        sum_2 += b*c/n
        Q += (a*b) / n
        R += (c*d) / n
        V += (a +b)* (c+d)*(a+c)*(b+d)/ (n*n*(n-1))

    se = sqrt(V/(Q*R))
    # se = sqrt(1/a+1/b+1/c+1/d)
    ef = exp(1.96*se)
    o1 = sum_1/sum_2
    o2 = sum_2/sum_1
    return [
        (o1, o1/ef, o1*ef), 
        (o2, o2/ef, o2*ef,)
        ]

    
#confidence interval
#https://www.researchgate.net/post/How_to_calculate_the_the_confidence_intervals_for_a_Mantel_Haenszel_stratified_Odds_ratio


In [ ]:
# 0: Facultad
# 1: Exámen aprobado
# 2: Preparación
cube = [
    [[414, 53], [37, 11]],
    [[20, 1], [135, 5]]
]
marginals(summary(cube))

print(cochran_mantel_haenzsel(cube))
print(ods_ratio(cube))


[[434, 54], [172, 16]]
(4.293676256123712, np.float64(0.03825436495200602))
[(2.0368647565451687, 0.22618135075032014, 18.34288292422331), (0.490950612595483, 0.05451705733123425, 4.421231001948929)]
